In [201]:
import torch
import torch.nn as nn
from torch.distributions import Normal, MixtureSameFamily, Categorical

# Generate Data

In [266]:
Categorical(torch.tensor([.1,.9]))

tensor([-2.3026, -0.1054])

In [301]:
X = torch.tensor(4.95)
mix = Categorical(torch.tensor([.1,.9]))
comp = Normal(torch.tensor([5,100]), torch.tensor([.5, 10]))
gmm = MixtureSameFamily(mix, comp)
# torch.exp(gmm.log_prob(X))
torch.exp(gmm.component_distribution.log_prob(X))

tensor([7.9391e-01, 9.6103e-22])

: 

In [299]:
X = torch.tensor(5.)
# torch.exp(Normal(torch.tensor([5,100]), torch.tensor([.1, 10])).log_prob(X))
torch.exp(Normal(torch.tensor(5.), torch.tensor(.01)).log_prob(X))

tensor(39.8942)

In [239]:

dist = Normal(torch.Tensor([10]), torch.Tensor([3]))
X = dist.sample((10,)).double()
X

tensor([[15.9911],
        [ 8.8948],
        [ 6.6468],
        [14.0990],
        [14.1473],
        [10.1454],
        [ 9.3467],
        [ 6.2313],
        [ 6.1152],
        [16.6622]], dtype=torch.float64)

In [240]:
class CustomLayer(nn.Module):
    def __init__(self): 
        super().__init__()

    def forward(self, input_features):
        output = torch.stack([input_features[:,0], torch.exp(input_features[:,1])], dim=1)
        return output

input_layer, out_layer, mid_layer = 1, 2, 10
seletor = nn.Sequential(
    nn.Linear(input_layer, mid_layer).double(),
    nn.ReLU(),
    nn.Linear(mid_layer, mid_layer).double(),
    nn.ReLU(),
    nn.Linear(mid_layer, out_layer).double(),
    CustomLayer()
)

In [261]:
def train(model, data, epochs=1000, log=False):
    optim = torch.optim.SGD(list(model.parameters()), lr=1e-4, momentum=.9)
    model.train(True)
    
    for epoch in range(epochs):
        optim.zero_grad()
        outputs = model(data)

        # loss
        mix = Categorical(torch.ones(outputs.shape[0]))
        comp = Normal(outputs[:,0], outputs[:,1])
        gmm = MixtureSameFamily(mix, comp)
        loss = -torch.sum(gmm.log_prob(data))
        ##
        
        if log:
            print(epoch, loss.item())

        loss.backward()
        optim.step() 
    model.train(False)
    return loss


loss = train(model=seletor, data=X)
print(loss)
est = seletor(X)
print(est)
# print(torch.exp(est[:,1]))


tensor(27.7175, dtype=torch.float64, grad_fn=<NegBackward0>)
tensor([[10.8121,  3.8683],
        [10.8121,  3.8683],
        [10.8121,  3.8683],
        [10.8121,  3.8683],
        [10.8121,  3.8683],
        [10.8121,  3.8683],
        [10.8121,  3.8683],
        [10.8121,  3.8683],
        [10.8121,  3.8683],
        [10.8121,  3.8683]], dtype=torch.float64, grad_fn=<StackBackward0>)


In [200]:
from torch.distributions import Normal, MixtureSameFamily, Categorical

x = torch.tensor(0)
# x = torch.tensor(10)
# x = torch.Tensor([0])
# x = torch.Tensor([0, 10, 12, -1, 5])
# x = torch.Tensor([[0, 10, 12, -1], [0, 10, 12, -1]])
# x = torch.Tensor([[0, 0], [10, 10], [12,12], [-1,-1]])

mix = Categorical(torch.Tensor([.1, .7]))
# mix = Categorical(torch.Tensor([.7, .7]))
comp = Normal(torch.Tensor([0, 10]), torch.Tensor([1, 1]))
gmm = MixtureSameFamily(mix, comp)

torch.exp(gmm.log_prob(x))

tensor(0.0499)

In [193]:
gmm.cdf(x)

tensor(0.2500)

In [168]:
torch.Tensor(10, 1).normal_(0, 1)

tensor([[ 0.3808],
        [-0.9332],
        [ 0.9977],
        [-0.4279],
        [ 0.1079],
        [-0.2924],
        [-1.2092],
        [-0.3668],
        [-0.6174],
        [-0.6691]])

In [185]:
from torch.distributions import Normal, Categorical, MixtureSameFamily

# Defina os parâmetros da mistura gaussiana
num_components = 3
mixture_probs = torch.tensor([0.3, 0.4, 0.3])  # probabilidades das componentes
component_means = torch.tensor([0.0, 2.0, 4.0])  # médias das componentes
component_stddevs = torch.tensor([1.0, 0.5, 1.5])  # desvios padrão das componentes

# Crie as distribuições normais para as componentes
component_distributions = Normal(component_means, component_stddevs)

# Crie a distribuição categórica para as misturas
mixture_distribution = Categorical(mixture_probs)

# Crie a mistura de distribuições
gaussian_mixture = MixtureSameFamily(mixture_distribution, component_distributions)

# Valor para calcular a probabilidade
value = torch.tensor(1.5)

# Calcule a log-probabilidade do valor para cada componente
log_probs = gaussian_mixture.log_prob(value)

# Converta a log-probabilidade para probabilidade
probs = torch.exp(log_probs)

print("Probabilidades de pertencer a cada componente:")
for i in range(num_components):
    print(f"Componente {i+1}: {probs}")

Probabilidades de pertencer a cada componente:
Componente 1: 0.2523272931575775
Componente 2: 0.2523272931575775
Componente 3: 0.2523272931575775
